Careless responder feature engineering and analysis

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
import scipy as sp
import math

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/final_data_complete.pkl'

In [ ]:
#read in csv containing data from all surveys
full_data = pd.read_pickle(data_path)
full_data = pd.DataFrame(data = full_data)
full_data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', full_data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', full_data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', full_data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', full_data.info(), '\n')

In [ ]:
full_data['survey_type'].value_counts()

In [ ]:
#should be 71
len(full_data['wave_study_day'].unique())

In [ ]:
#create study date bins
full_data['wave_study_date_bin'] = pd.cut(full_data['wave_study_day'], 5)
full_data['wave_study_date_bin'].value_counts()

In [ ]:
full_data.head()

## CR Functions

In [ ]:
def context_homevsworking_CR(row, location_col, activity_col):
    
    #if at home should not be working
    if (row[location_col] == num_dic['location_home']) & (row[activity_col] == num_dic['activity_work']):
        context_homevsworking = 1
    else:
        context_homevsworking = 0

    return context_homevsworking

In [ ]:
def context_workvsactivities_CR(row, location_col, activity_col):
    
    #if at work should not be playing sports, household activities, civic duties
    if (row[location_col] == num_dic['location_work']) & \
    ((row[activity_col] == num_dic['activity_sports']) | (row[activity_col] == num_dic['activity_household']) | (row[activity_col] == num_dic['activity_civic'])):
        context_workvsactivities = 1
    else:
        context_workvsactivities = 0

    return context_workvsactivities

In [ ]:
def context_workvswork_CR(row, location_col, activity_col):
    
    #if at work should be working
    if (row[location_col] == num_dic['location_work']) & (row[activity_col] != num_dic['activity_work']):
        context_workvswork = 1
    else:
        context_workvswork = 0

    return context_workvswork

In [ ]:
def context_drivevsdrive_CR(row, location_col, activity_col):
    
    #if at vehicle should be driving/travel    
    if (row[location_col] == num_dic['location_vehicle']) & (row[activity_col] != num_dic['activity_drive']):
        context_drivevsdrive = 1
    else:
        context_drivevsdrive = 0

    return context_drivevsdrive

In [ ]:
def mean_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].dropna().values
    b = row.loc[param3:param4].dropna().values
    
    if (len(a) | len(b)) == 0.0:
        mean_diff = np.nan
    else:
        try:
            mean_diff = np.abs(np.mean(a) -  np.mean(b))
        except:
            mean_diff = np.nan
            
    return mean_diff

In [ ]:
def median_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].values
    b = row.loc[param3:param4].values
    
    if (len(a) | len(b)) == 0.0:
        median_diff = np.nan
    else:
        try:
            median_diff = np.abs(np.median(a) -  np.median(b))
        except:
            median_diff = np.nan
        
    return median_diff

In [ ]:
def mode_diff(row, param1, param2, param3, param4):
    
    a = row.loc[param1:param2].values
    b = row.loc[param3:param4].values
    
    if (len(a) | len(b)) == 0.0:
        mode_diff = np.nan
    else:
        try:
            mode_diff = np.abs(sp.stats.mode(a)[0][0] -  sp.stats.mode(b)[0][0])
        except:
            mode_diff = np.nan
    
    return mode_diff

In [ ]:
def longstring_CR(row, num_questions):
    #create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

    groups = groupby(row)
    result = [(label, sum(1 for _ in group)) for label, group in groups]
    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    if max_string_length == 1:
        max_answer = 0
        
    max_string_length_norm = max_string_length / num_questions
    
    return pd.Series((max_string_length, max_answer, max_string_length_norm))

In [ ]:
def moments_CR(row):
    
    #auc
    auc = np.trapz(row.dropna())
    
    #std
    std = np.std(row.dropna())
    
    #skew
    if std == 0.0:
        skew = 0.0
    else:
        try:
            skew = sp.stats.skew(row.dropna())
        except:
            skew = np.nan
    
    #kurtosis
    if std == 0.0:
        kurt = -3.0
    else:
        try:
            kurt = sp.stats.kurtosis(row.dropna())
        except:
            kurt = np.nan
            
    return pd.Series((auc, std, skew, kurt))

In [ ]:
def moments_seeded_CR(row):
    
    #auc
    auc_seeded = np.trapz(np.append(row.dropna().values, 0.0))
    
    #std
    std_seeded = np.std(np.append(row.dropna().values, 0.0))

    #skew
    if std_seeded == 0.0:
        skew_seeded = 0.0
    else:
        try:
            skew_seeded = sp.stats.skew(np.append(row.dropna().values, 0.0))
        except:
            skew_seeded = np.nan
    
    #kurtosis
    if std_seeded == 0.0:
        kurt_seeded = -3.0
    else:
        try:
            kurt_seeded = sp.stats.kurtosis(np.append(row.dropna().values, 0.0))
        except:
            kurt_seeded = np.nan
            
    return pd.Series((auc_seeded, std_seeded, skew_seeded, kurt_seeded))

In [ ]:
def intake_check_CR(row, param1, param2, param3):
    
    #If intake = yes, then SUM(intake types) must be > 0
    if (row[param1] == 1) & (row.loc[param2:param3].sum() <= 0):
        intake_check = 1
    else:
        intake_check = 0

    return intake_check

## CR feature creation

### notes for CR features for Engage surveys

Context question
- Semantic Antonyms
    - if context1 = home (0), then context2 ≠ work and work related (0)
    - if context1 = work (1), then context2 ≠ leisure sports (4), household activities (7), org/civic (11)
- Semantic Synonyms
    - if context1 = work (1), then context2 most likely work and work related (0)
    - If context1 = vehicle (4), then context2 most likely travel or commute (12)
- Internal consistency
    - if context1 = 5 (other) then should have a write in
    - if context2 = 13 (other) then should have a write in

Longstring
- All questions use same scale (1=not at all, 7=very much), but there are 5 different constructs assessed

Semantic consistency
- Internal consistency (within construct) should be greater than consistency across constructs

Semantic synonyms 
- not applicable 

Semantic antonyms
- Hindrance stressors should be negatively correlated with support 


In [ ]:
#split off completed engage and related columns
engage_only = full_data[(full_data['survey_type'] == 'engage_psycap') & (full_data['completed'] == 1.0)]

print(engage_only.shape)
engage_only['ParticipantID'].unique().shape

In [ ]:
%%time

num_dic = {'location_home': 0, 'location_work': 1, 'location_vehicle': 4,
           'activity_work': 0, 'activity_sports': 4, 'activity_household': 7, 'activity_civic': 11, 'activity_drive': 12}
location_col = 'location_num'
activity_col = 'activity_num'
engage_only['context_homevsworking'] = engage_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
engage_only['context_workvsactivities'] = engage_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
engage_only['context_workvswork'] = engage_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
engage_only['context_drivevsdrive'] = engage_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#support_mgt should be negatively correlated with hindrance_mgt
param1 = 'support_mgt'
param2 = 'support_mgt'
param3 = 'hindrance_mgt'
param4 = 'hindrance_mgt'
engage_only['mean_diff_hinderance_vs_support'] = engage_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#engage_only['meadian_diff_hinderance_vs_support'] = engage_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#engage_only['mode_diff_hinderance_vs_support'] = engage_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

param1 = 'engage_3'
param2 = 'engage_29'
num_questions = 27
engage_only[['longstring_count_engage', 'longstring_answer_engage', 'longstring_count_norm_engage']] = engage_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
engage_only['longstring_mult_engage'] = engage_only['longstring_count_engage'] * engage_only['longstring_answer_engage']

engage_only[['ls_auc_engage', 'ls_std_engage', 'ls_skew_engage', 'ls_kurt_engage']] = engage_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
engage_only[['ls_auc_seeded_engage', 'ls_std_seeded_engage', 'ls_skew_seeded_engage', 'ls_kurt_seeded_engage']] = engage_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

engage_only['longstring_count_norm_ave'] = engage_only['longstring_count_norm_engage']

### notes for CR features for Psych Flex

Should have answered every question

Longstring
- Legitimate longstrings of  ≥ 8 are unlikely for response “5”
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Legitimate scores of pf_mgt=5 are almost impossible

Semantic antonyms
- Not applicable

Semantic synonyms 
- not applicable 


In [ ]:
#split off completed PF and related columns
psych_flex_only = full_data[(full_data['survey_type'] == 'psych_flex') & (full_data['completed'] == 1.0)]

print(psych_flex_only.shape)
psych_flex_only['ParticipantID'].unique().shape

In [ ]:
param1 = 'pf_03'
param2 = 'pf_15'
num_questions = 13

psych_flex_only[['longstring_count_pf', 'longstring_answer_pf', 'longstring_count_norm_pf']] = psych_flex_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
psych_flex_only['longstring_mult_pf'] = psych_flex_only['longstring_count_pf'] * psych_flex_only['longstring_answer_pf']

psych_flex_only[['ls_auc_pf', 'ls_std_pf', 'ls_skew_pf', 'ls_kurt_pf']] = psych_flex_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
psych_flex_only[['ls_auc_seeded_pf', 'ls_std_seeded_pf', 'ls_skew_seeded_pf', 'ls_kurt_seeded_pf']] = psych_flex_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

psych_flex_only['longstring_count_norm_ave'] = psych_flex_only['longstring_count_norm_pf']

### notes for CR features for Jobs

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - IRB questions use same scale
    - dalal questions use same scale
- Consistency
    - irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
    - itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
    - dalal1-8 should be negatively correlated with dalal9-dalal16



In [ ]:
#split off completed (eg at work) job and related columns
jobs_atwork_only = full_data[(full_data['survey_type'] == 'job') & (full_data['work'] == 1)]

print(jobs_atwork_only.shape)
jobs_atwork_only['ParticipantID'].unique().shape

In [ ]:
%%time

#update survey type column to reflect if at work or not
jobs_atwork_only['survey_type'] = 'jobs_atwork'

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
jobs_atwork_only['context_homevsworking'] = jobs_atwork_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_workvsactivities'] = jobs_atwork_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_workvswork'] = jobs_atwork_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
jobs_atwork_only['context_drivevsdrive'] = jobs_atwork_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
jobs_atwork_only['mean_diff_affect_posneg'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['median_diff_affect_posneg'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_affect_posneg'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
jobs_atwork_only['mean_diff_nervous_anxiety'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['median_diff_nervous_anxiety'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_nervous_anxiety'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
  
#irb1, irb2, irb3, irb4 should be negatively correlated with irb6 and irb7
param1 = 'irbd1'
param2 ='irbd4'
param3 = 'irbd6'
param4 = 'irbd7'
jobs_atwork_only['mean_diff_irb_irb'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['median_diff_irb_irb'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_irb_irb'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#itp1, itp2, itp3 should be negatively correlated with irb6 and irb7
param1 = 'itpd1'
param2 ='itpd3'
param3 = 'irbd6'
param4 = 'irbd7'
jobs_atwork_only['mean_diff_irb_itp'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['median_diff_irb_itp'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_irb_itp'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#dalal1-8 should be negatively correlated with dalal9-dalal16
param1 = 'dalal1'
param2 ='dalal8'
param3 = 'dalal9'
param4 = 'dalal16'
jobs_atwork_only['mean_diff_dalal_posneg'] = jobs_atwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['median_diff_dalal_posneg'] = jobs_atwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_atwork_only['mode_diff_dalal_posneg'] = jobs_atwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
jobs_atwork_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_atwork_only['longstring_mult_affect'] = jobs_atwork_only['longstring_count_affect'] * jobs_atwork_only['longstring_answer_affect']
jobs_atwork_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#irbd longstring
param1 = 'irbd1'
param2 = 'irbd7'
num_questions = 7
jobs_atwork_only[['longstring_count_irbd', 'longstring_answer_irbd', 'longstring_count_norm_irbd']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_atwork_only['longstring_mult_irbd'] = jobs_atwork_only['longstring_count_irbd'] * jobs_atwork_only['longstring_answer_irbd']
jobs_atwork_only[['ls_auc_irbd', 'ls_std_irbd', 'ls_skew_irbd', 'ls_kurt_irbd']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['ls_auc_seeded_irbd', 'ls_std_seeded_irbd', 'ls_skew_seeded_irbd', 'ls_kurt_seeded_irbd']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#dalal longstring
param1 = 'dalal1'
param2 = 'dalal16'
num_questions = 16
jobs_atwork_only[['longstring_count_dalal', 'longstring_answer_dalal', 'longstring_count_norm_dalal']] = jobs_atwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_atwork_only['longstring_mult_dalal'] = jobs_atwork_only['longstring_count_dalal'] * jobs_atwork_only['longstring_answer_dalal']
jobs_atwork_only[['ls_auc_dalal', 'ls_std_dalal', 'ls_skew_dalal', 'ls_kurt_dalal']] = jobs_atwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_atwork_only[['ls_auc_seeded_dalal', 'ls_std_seeded_dalal', 'ls_skew_seeded_dalal', 'ls_kurt_seeded_dalal']] = jobs_atwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#find ave longstring norm
jobs_atwork_only['longstring_count_norm_ave'] = jobs_atwork_only[['longstring_count_norm_affect', 'longstring_count_norm_irbd', 'longstring_count_norm_dalal']].mean(axis=1)

In [ ]:
#split off non at work job and related columns
jobs_notatwork_only = full_data[(full_data['survey_type'] == 'job') & (full_data['work'] == 2)]

print(jobs_notatwork_only.shape)
jobs_notatwork_only['ParticipantID'].unique().shape

In [ ]:
%%time

#update survey type column to reflect if at work or not
jobs_notatwork_only['survey_type'] = 'jobs_notatwork'

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
jobs_notatwork_only['context_homevsworking'] = jobs_notatwork_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
jobs_notatwork_only['context_workvsactivities'] = jobs_notatwork_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
jobs_notatwork_only['context_workvswork'] = jobs_notatwork_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
jobs_notatwork_only['context_drivevsdrive'] = jobs_notatwork_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
jobs_notatwork_only['mean_diff_affect_posneg'] = jobs_notatwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_notatwork_only['median_diff_affect_posneg'] = jobs_notatwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_notatwork_only['mode_diff_affect_posneg'] = jobs_notatwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
jobs_notatwork_only['mean_diff_nervous_anxiety'] = jobs_notatwork_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#jobs_notatwork_only['median_diff_nervous_anxiety'] = jobs_notatwork_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#jobs_notatwork_only['mode_diff_nervous_anxiety'] = jobs_notatwork_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
jobs_notatwork_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = jobs_notatwork_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
jobs_notatwork_only['longstring_mult_affect'] = jobs_notatwork_only['longstring_count_affect'] * jobs_notatwork_only['longstring_answer_affect']
jobs_notatwork_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = jobs_notatwork_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
jobs_notatwork_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = jobs_notatwork_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

jobs_notatwork_only['longstring_count_norm_ave'] = jobs_notatwork_only['longstring_count_norm_affect']

### notes for CR features for Health

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - n/a
- Consistency
    - If alc1 = yes, then SUM(alc2_1, alc2_2, alc2_3) must be > 0
    - If tob1 = yes, then SUM(tob2_1, tob2_2, tob2_3, tob2_4, tob2_5, tob2_6, tob2_7) must be > 0


In [ ]:
#split off completed job and related columns
health_only = full_data[(full_data['survey_type'] == 'health')]

print(health_only.shape)
health_only['ParticipantID'].unique().shape

In [ ]:
%%time

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
health_only['context_homevsworking'] = health_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
health_only['context_workvsactivities'] = health_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
health_only['context_workvswork'] = health_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
health_only['context_drivevsdrive'] = health_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
health_only['mean_diff_affect_posneg'] = health_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#health_only['median_diff_affect_posneg'] = health_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#health_only['mode_diff_affect_posneg'] = health_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
health_only['mean_diff_nervous_anxiety'] = health_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#health_only['median_diff_nervous_anxiety'] = health_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#health_only['mode_diff_nervous_anxiety'] = health_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
  
#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
health_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = health_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
health_only['longstring_mult_affect'] = health_only['longstring_count_affect'] * health_only['longstring_answer_affect']
health_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = health_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
health_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = health_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

health_only['longstring_count_norm_ave'] = health_only['longstring_count_norm_affect']

#alc consistency
param1 = 'alc1'
param2 = 'alc2_1'
param3 = 'alc2_3'
health_only['intake_check_alc'] = health_only.apply(lambda row: intake_check_CR(row, param1, param2, param3), axis=1)

#nicotine consistency
param1 = 'tob1'
param2 = 'tob2_1'
param3 = 'tob2_7'
health_only['intake_check_nic'] = health_only.apply(lambda row: intake_check_CR(row, param1, param2, param3), axis=1)


### notes for CR features for Personality

Context question (context 2 = activity, context 3 = location)
- Semantic Antonyms
    - if context3 = home (1), then context2 ≠ work and work related (1)
    - if context3 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)
- Semantic Synonyms
    - if context3 = work (2), then context2 most likely work and work related (1)
    - If context3 = vehicle (5), then context2 most likely travel or commute (11)

Affect/Anxiety/Stress
- Longstrings
    - All questions use same scale
- Semantic antonyms
    - Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)

Task Perfomrance
- Longstrings
    - All questions use same scale
- Consistency - each pair should be negatively correlated (eg each pair is reverse scored)
    - bfi1 and bfi6 (extraversion)
    - bfi2 and bfi7 (agreeableness)
    - bfi3 and bfi8 (conscientiousness)
    - bfi4 and bfi9 (neuroticism)
    - bfi5 and bfi10 (openness)


In [ ]:
#split off completed job and related columns
personality_only = full_data[(full_data['survey_type'] == 'personality')]

print(personality_only.shape)
personality_only['ParticipantID'].unique().shape

In [ ]:
%%time

#context comparisons
num_dic = {'location_home': 1, 'location_work': 2, 'location_vehicle': 5,
           'activity_work': 1, 'activity_sports': 3, 'activity_household': 6, 'activity_civic': 10, 'activity_drive': 11}
location_col = 'context3'
activity_col = 'context2'
personality_only['context_homevsworking'] = personality_only.apply(lambda row: context_homevsworking_CR(row, location_col, activity_col), axis=1)
personality_only['context_workvsactivities'] = personality_only.apply(lambda row: context_workvsactivities_CR(row, location_col, activity_col), axis=1)
personality_only['context_workvswork'] = personality_only.apply(lambda row: context_workvswork_CR(row, location_col, activity_col), axis=1)
personality_only['context_drivevsdrive'] = personality_only.apply(lambda row: context_drivevsdrive_CR(row, location_col, activity_col), axis=1)

#Positive block (pan1-5) should be negatively correlated with negative block (pan6-10)
param1 = 'pand1'
param2 ='pand5'
param3 = 'pand6'
param4 = 'pand10'
personality_only['mean_diff_affect_posneg'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['median_diff_affect_posneg'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_affect_posneg'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
 
#pand10 (nervousness) should be positivitely correlated with anxiety
param1 = 'pand10'
param2 ='pand10'
param3 = 'anxiety'
param4 = 'anxiety'
personality_only['mean_diff_nervous_anxiety'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['median_diff_nervous_anxiety'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_nervous_anxiety'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)
    
#bfi1 and bfi6 (extraversion) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid1'
param2 ='bfid1'
param3 = 'bfid6'
param4 = 'bfid6'
personality_only['mean_diff_extraversion'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['median_diff_extraversion'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_extraversion'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi2 and bfi7 (agreeableness) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid2'
param2 ='bfid2'
param3 = 'bfid7'
param4 = 'bfid7'
personality_only['mean_diff_agreeableness'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['median_diff_agreeableness'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_agreeableness'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi3 and bfi8 (conscientiousness) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid3'
param2 ='bfid3'
param3 = 'bfid8'
param4 = 'bfid8'
personality_only['mean_diff_conscientiousness'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['median_diff_conscientiousness'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_conscientiousness'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi4 and bfi9 (neuroticism) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid4'
param2 ='bfid4'
param3 = 'bfid9'
param4 = 'bfid9'
personality_only['mean_diff_neuroticism'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['median_diff_neuroticism'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_neuroticism'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)

#bfi5 and bfi10 (openness) should be negatively correlated (eg each pair is reverse scored)
param1 = 'bfid5'
param2 ='bfid5'
param3 = 'bfid10'
param4 = 'bfid10'
personality_only['mean_diff_openness'] = personality_only.apply(lambda row: mean_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['median_diff_openness'] = personality_only.apply(lambda row: median_diff(row, param1, param2, param3, param4), axis=1)
#personality_only['mode_diff_openness'] = personality_only.apply(lambda row: mode_diff(row, param1, param2, param3, param4), axis=1)


#affect longstring
param1 = 'pand1'
param2 = 'pand10'
num_questions = 10
personality_only[['longstring_count_affect', 'longstring_answer_affect', 'longstring_count_norm_affect']] = personality_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
personality_only['longstring_mult_affect'] = personality_only['longstring_count_affect'] * personality_only['longstring_answer_affect']
personality_only[['ls_auc_affect', 'ls_std_affect', 'ls_skew_affect', 'ls_kurt_affect']] = personality_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
personality_only[['ls_auc_seeded_affect', 'ls_std_seeded_affect', 'ls_skew_seeded_affect', 'ls_kurt_seeded_affect']] = personality_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#personality longstring
param1 = 'bfid1'
param2 = 'bfid10'
num_questions = 10
personality_only[['longstring_count_personality', 'longstring_answer_personality', 'longstring_count_norm_personality']] = personality_only.apply(lambda row: longstring_CR(row.loc[param1:param2], num_questions), axis=1)
personality_only['longstring_mult_personality'] = personality_only['longstring_count_personality'] * personality_only['longstring_answer_affect']
personality_only[['ls_auc_personality', 'ls_std_personality', 'ls_skew_personality', 'ls_kurt_personality']] = personality_only.apply(lambda row: moments_CR(row.loc[param1:param2]), axis=1)
personality_only[['ls_auc_seeded_personality', 'ls_std_seeded_personality', 'ls_skew_seeded_personality', 'ls_kurt_seeded_personality']] = personality_only.apply(lambda row: moments_seeded_CR(row.loc[param1:param2]), axis=1)

#find ave longstring norm
personality_only['longstring_count_norm_ave'] = personality_only[['longstring_count_norm_affect', 'longstring_count_norm_personality']].mean(axis=1)

In [ ]:
surveys_CR = pd.concat([engage_only, psych_flex_only, jobs_atwork_only, jobs_notatwork_only, health_only, personality_only], axis = 0, sort=False)
surveys_CR.to_csv('surveys_CR.csv')
surveys_CR.head()

### explore CR across survey types

In [ ]:
surveys_CR['survey_type'].value_counts()

In [ ]:
shared_cols = ['context_homevsworking',
       'context_workvsactivities', 'context_workvswork',
       'context_drivevsdrive', 'mean_diff_affect_posneg', 'mean_diff_nervous_anxiety',
               'longstring_count_affect', 'longstring_answer_affect',
       'longstring_mult_affect', 'longstring_count_norm_affect', 'longstring_count_norm_ave', 'ls_auc_affect', 'ls_std_affect',
       'ls_skew_affect', 'ls_kurt_affect', 'ls_auc_seeded_affect',
       'ls_std_seeded_affect', 'ls_skew_seeded_affect',
       'ls_kurt_seeded_affect']

engage_specific = ['mean_diff_hinderance_vs_support',
       'longstring_count_engage', 'longstring_answer_engage',
       'longstring_mult_engage', 'ls_auc_engage', 'ls_std_engage',
       'ls_skew_engage', 'ls_kurt_engage', 'ls_auc_seeded_engage',
       'ls_std_seeded_engage', 'ls_skew_seeded_engage',
       'ls_kurt_seeded_engage']

pf_specific = [ 'longstring_count_pf',
       'longstring_answer_pf', 'longstring_mult_pf', 'ls_auc_pf',
       'ls_std_pf', 'ls_skew_pf', 'ls_kurt_pf', 'ls_auc_seeded_pf',
       'ls_std_seeded_pf', 'ls_skew_seeded_pf', 'ls_kurt_seeded_pf']

jobs_specific = ['mean_diff_irb_irb', 'mean_diff_irb_itp', 'mean_diff_dalal_posneg',
     'longstring_count_irbd', 'longstring_answer_irbd', 'longstring_mult_irbd', 'ls_auc_irbd',
       'ls_std_irbd', 'ls_skew_irbd', 'ls_kurt_irbd',
       'ls_auc_seeded_irbd', 'ls_std_seeded_irbd', 'ls_skew_seeded_irbd',
       'ls_kurt_seeded_irbd', 'longstring_count_dalal',
       'longstring_answer_dalal', 'longstring_mult_dalal', 'ls_auc_dalal',
       'ls_std_dalal', 'ls_skew_dalal', 'ls_kurt_dalal',
       'ls_auc_seeded_dalal', 'ls_std_seeded_dalal',
       'ls_skew_seeded_dalal', 'ls_kurt_seeded_dalal']

health_specific =  ['intake_check_alc',
       'intake_check_nic']

personality_specific = ['mean_diff_extraversion',
       'mean_diff_agreeableness', 'mean_diff_conscientiousness',
       'mean_diff_neuroticism', 'mean_diff_openness',
       'longstring_count_personality', 'longstring_answer_personality',
       'longstring_mult_personality', 'ls_auc_personality',
       'ls_std_personality', 'ls_skew_personality', 'ls_kurt_personality',
       'ls_auc_seeded_personality', 'ls_std_seeded_personality',
       'ls_skew_seeded_personality', 'ls_kurt_seeded_personality']

In [ ]:
i=1
plt.figure(figsize=(50,30))
grid_size = math.ceil(np.sqrt(len(shared_cols)))
for feature in shared_cols:
    plt.subplot(grid_size, grid_size, i)
    sns.barplot(x='survey_type', y=feature, data=surveys_CR)
    i = i + 1

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='wave_study_day', y='mean_diff_affect_posneg', data=surveys_CR, hue='survey_type', ci=None)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='wave_study_date_bin', y='mean_diff_affect_posneg', data=surveys_CR, hue='survey_type', ci=68)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='wave_study_date_bin', y='longstring_count_norm_ave', data=surveys_CR, hue='survey_type', ci=68)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='wave_study_date_bin', y='longstring_count_norm_affect', data=surveys_CR, hue='survey_type', ci=68)

In [ ]:
print(engage_only.shape)
engage_only_CRs = engage_only[engage_cols].dropna(axis=0)
engage_only_CRs.set_index('MitreID', inplace=True)
print(engage_only_CRs.shape)
print(engage_only_CRs.isna().sum())

In [ ]:
print(psych_flex_only.shape)
psych_flex_only_CRs = psych_flex_only[psych_flex_cols].dropna(axis=0)
psych_flex_only_CRs.set_index('MitreID', inplace=True)
print(psych_flex_only_CRs.shape)
print(psych_flex_only_CRs.isna().sum())

In [ ]:
print(jobs_atwork_only.shape)
jobs_atwork_only_CRs = jobs_atwork_only[jobs_atwork_only_cols].dropna(axis=0)
jobs_atwork_only_CRs.set_index('MitreID', inplace=True)
print(jobs_atwork_only_CRs.shape)
print(jobs_atwork_only_CRs.isna().sum())

In [ ]:
engage_CRs_corr = engage_only_CRs.corr()
engage_CRs_corr.to_csv('engage_CRs_corr.csv')
engage_CRs_corr

In [ ]:
psych_flex_CRs_corr = psych_flex_only_CRs.corr()
psych_flex_CRs_corr.to_csv('psych_flex_CRs_corr.csv')
psych_flex_CRs_corr

In [ ]:
jobs_atwork_CRs_corr = jobs_atwork_only_CRs.corr()
#jobs_atwork_CRs_corr.to_csv('jobs_atwork_CRs_corr.csv')
jobs_atwork_CRs_corr

In [ ]:
i=1
plt.figure(figsize=(30,20))
grid_size = math.ceil(np.sqrt(len(engage_cols)))
for feature in engage_cols[1:]:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(engage_only_CRs[feature])
    i = i + 1


In [ ]:
i=1
plt.figure(figsize=(30,20))
grid_size = math.ceil(np.sqrt(len(psych_flex_cols)))
for feature in psych_flex_cols[1:]:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(psych_flex_only_CRs[feature])
    i = i + 1


In [ ]:
i=1
plt.figure(figsize=(30,30))
grid_size = math.ceil(np.sqrt(len(jobs_atwork_only_cols)))
for feature in jobs_atwork_only_cols[1:]:
    plt.subplot(grid_size, grid_size, i)
    sns.distplot(jobs_atwork_only_CRs[feature])
    i = i + 1


In [ ]:
sns.pairplot(jobs_atwork_only_CRs[['mean_diff_affect_posneg',
       'mean_diff_nervous_anxiety', 'mean_diff_irb_irb',
       'mean_diff_irb_itp', 'mean_diff_dalal_posneg']], kind='reg', diag_kind="kde")

In [ ]:
sns.pairplot(jobs_atwork_only_CRs[['longstring_count_affect',
       'longstring_mult_affect', 'longstring_count_irbd',
       'longstring_mult_irbd', 'longstring_count_dalal', 'longstring_mult_dalal']], kind='reg', diag_kind="kde")

In [ ]:
for feature in jobs_atwork_only_cols:
    sns.jointplot(jobs_CRs['irb_irb_mean'], jobs_CRs[feature], kind='hex')
    plt.show()

## Clustering

In [ ]:
jobs_atwork_only_CRs[['longstring_count_affect', 'longstring_count_irbd', 'longstring_count_dalal']].corr().sum()

In [ ]:
engage_select_features = ['mean_diff_hinderance_vs_support',
       'longstring_count', 'longstring_mult', 'ls_auc', 'ls_std',
       'ls_skew_seeded', 'ls_kurt_seeded', 'time_to_complete']

psych_select_features = ['longstring_count', 'longstring_mult', 'ls_auc', 'ls_std',
       'ls_skew_seeded', 'ls_kurt_seeded', 'time_to_complete']

jobs_select_features = ['mean_diff_irb_itp', 
                    'longstring_count_affect', 
                    'longstring_mult_affect',
                    'ls_std_affect', 
                    'ls_skew_seeded_affect', 
                    'ls_kurt_seeded_affect', 
                    'time_to_complete']

In [ ]:
# center and scale the data
scaler = StandardScaler()

#full feature set
features_engage_full_scaled = scaler.fit_transform(engage_only_CRs)
features_psychflex_full_scaled = scaler.fit_transform(psych_flex_only_CRs)
features_jobs_atwork_full_scaled = scaler.fit_transform(jobs_atwork_only_CRs)

#select feature set
features_engage_select = engage_only_CRs[engage_select_features]
features_psychflex_select = psych_flex_only_CRs[psych_select_features]
features_jobs_atwork_select = jobs_atwork_only_CRs[jobs_select_features]

features_engage_select_scaled = scaler.fit_transform(engage_only_CRs)
features_psychflex_select_scaled = scaler.fit_transform(psych_flex_only_CRs)
features_jobs_atwork_select_scaled = scaler.fit_transform(jobs_atwork_only_CRs)

In [ ]:
k_range = range(2,10)
scores = []
for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(features_jobs_atwork_full_scaled)
    scores.append(silhouette_score(features_jobs_atwork_select_scaled, km_ss.labels_))

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('PF kmeans at survey level')

In [ ]:
engage_km_survey = KMeans(n_clusters=2,random_state=1234)
engage_km_survey.fit(engage_survey_features_scaled)
print(silhouette_score(engage_survey_features_scaled, engage_km_survey.labels_))

engage_only_features['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]
engage_only_features_cont['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]
engage_only['kmeans_scaled_survey'] = [label for label in engage_km_survey.labels_ ]

engage_only['kmeans_scaled_survey'].value_counts()

## TSNE 

In [ ]:
%%time

# This function can take a long to run
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
tsne_features_engage_full = tsne.fit_transform(features_engage_full_scaled)
tsne_features_psychflex_full = tsne.fit_transform(features_psychflex_full_scaled)
tsne_features_jobs_full = tsne.fit_transform(features_jobs_atwork_full_scaled)

In [ ]:
print(tsne_features_engage.shape)
tsne_features_engage_df = pd.DataFrame(data = tsne_features_engage_full, columns = ['tsne_0', 'tsne_1'], index = engage_only_CRs.index)
tsne_features_engage_df = pd.concat([engage_only_CRs, tsne_features_engage_df], axis = 1)

plt.figure(figsize=(7,7))
sns.scatterplot(x = 'tsne_0', y = 'tsne_1', data = tsne_features_engage_df)
plt.title("Projection of the data on 2 components + ground truth labels")

In [ ]:
print(tsne_features_psychflex.shape)
tsne_features_psychflex_df = pd.DataFrame(data = tsne_features_psychflex_full, columns = ['tsne_0', 'tsne_1'], index = psych_flex_only_CRs.index)
tsne_features_psychflex_df = pd.concat([psych_flex_only_CRs, tsne_features_psychflex_df], axis = 1)

plt.figure(figsize=(7,7))
sns.scatterplot(x = 'tsne_0', y = 'tsne_1', data = tsne_features_psychflex_df)
plt.title("Projection of the data on 2 components + ground truth labels")

In [ ]:
print(tsne_features_jobs.shape)
tsne_features_jobs_df = pd.DataFrame(data = tsne_features_jobs_full, columns = ['tsne_0', 'tsne_1'], index = jobs_atwork_only_CRs.index)
tsne_features_jobs_df = pd.concat([jobs_atwork_only_CRs, tsne_features_jobs_df], axis = 1)

plt.figure(figsize=(7,7))
sns.scatterplot(x = 'tsne_0', y = 'tsne_1', data = tsne_features_jobs_df)
plt.title("Projection of the data on 2 components + ground truth labels")

In [ ]:
%%time

# This function can take a long to run
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
tsne_features_engage_select = tsne.fit_transform(features_engage_select_scaled)
tsne_features_psychflex_select = tsne.fit_transform(features_psychflex_select_scaled)
tsne_features_jobs_select = tsne.fit_transform(features_jobs_atwork_select_scaled)

In [ ]:
print(tsne_features_engage.shape)
tsne_features_engage_df = pd.DataFrame(data = tsne_features_engage_select, columns = ['tsne_0', 'tsne_1'], index = engage_only_CRs.index)
tsne_features_engage_df = pd.concat([engage_only_CRs, tsne_features_engage_df], axis = 1)

plt.figure(figsize=(7,7))
sns.scatterplot(x = 'tsne_0', y = 'tsne_1', data = tsne_features_engage_df)
plt.title("Projection of the data on 2 components + ground truth labels")

In [ ]:
print(tsne_features_psychflex.shape)
tsne_features_psychflex_df = pd.DataFrame(data = tsne_features_psychflex_select, columns = ['tsne_0', 'tsne_1'], index = psych_flex_only_CRs.index)
tsne_features_psychflex_df = pd.concat([psych_flex_only_CRs, tsne_features_psychflex_df], axis = 1)

plt.figure(figsize=(7,7))
sns.scatterplot(x = 'tsne_0', y = 'tsne_1', data = tsne_features_psychflex_df)
plt.title("Projection of the data on 2 components + ground truth labels")

In [ ]:
print(tsne_features_jobs.shape)
tsne_features_jobs_df = pd.DataFrame(data = tsne_features_jobs_select, columns = ['tsne_0', 'tsne_1'], index = jobs_atwork_only_CRs.index)
tsne_features_jobs_df = pd.concat([jobs_atwork_only_CRs, tsne_features_jobs_df], axis = 1)

plt.figure(figsize=(7,7))
sns.scatterplot(x = 'tsne_0', y = 'tsne_1', data = tsne_features_jobs_df)
plt.title("Projection of the data on 2 components + ground truth labels")

In [ ]:
participants = engage_only['MitreID'].unique()

for part in participants:
    data_part = engage_only[engage_only['MitreID'] == part]
    sns.countplot(x='wave_study_day', hue='kmeans_scaled_survey', data=data_part)
    plt.show()

In [ ]:
#add cluster 1 % to final data
participants = data['MitreID'].unique()

for part in participants:
    if part in engage_only_features.index:
        perc = engage_only_features[(engage_only_features.index == part) & (engage_only_features['kmeans_scaled_survey'] == 1)].shape[0] / engage_only_features[engage_only_features.index == part].shape[0]

        data.loc[data['MitreID'] == part, 'engage_CR_perc'] = perc
        engage_only.loc[engage_only['MitreID'] == part, 'engage_CR_perc'] = perc
    
    else:
        data.loc[data['MitreID'] == part, 'engage_CR_perc'] = np.nan
        engage_only.loc[engage_only['MitreID'] == part, 'engage_CR_perc'] = np.nan